In [1]:
#use corv_wd_models.yml environment

from astropy.io import fits
from astropy.table import Table, Column, MaskedColumn, join, vstack
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from tqdm import tqdm

#to import SDSS-V data
import fsspec
import requests
import aiohttp

#to query Gaia
from astroquery.gaia import Gaia

#to import SDSS-IV data
from astroquery.sdss import SDSS


In [2]:
#input SDSS-V username and password
file=open('SDSSVlogin.txt')
content = file.readlines()
username=content[0][:len(content[0])-1] #remove the \n
password=content[1]

# This notebook gets the sample of all SDSS-V spectra flagged by SnowWhite as DA WDs for the IPL-3 v6_1_3 reductions

# Get Sample of SDSS-V Spectra

The data can be accessed from SAS: https://data.sdss5.org/sas/sdsswork/

You must use the common SDSS username and password

**LATEST link to data:**

https://data.sdss5.org/sas/ipl-3/spectro/boss/redux/v6_1_3/spectra/


The spectra directory is organized as follows:

    - Plate number (ex. 15000/)
        - MJD (ex. 15000/59146/)
            - spectum fits file labeled by spec-PLATE-MJD-CATALOGID.fits (ex. spec-015000-59146-4375786564.fits)
            - the most recent spAll file also has a SPEC_FILE column which gives the name of the fits file

Open the latest version of the SnowWhite summary file from astra. All summary files are available at https://data.sdss5.org/sas/sdsswork/mwm/spectro/astra/0.5.0/summary/

In [3]:
#to open the latest version of SnowWhite from the SDSS-V database without downloading it
file='astraAllVisitSnowWhite-0.5.0.fits.gz'

url='https://data.sdss5.org/sas/sdsswork/mwm/spectro/astra/0.5.0/summary/'+file

astra_SW_outputs=fits.open(url, use_fsspec=True, fsspec_kwargs={"auth":aiohttp.BasicAuth(username, password)})

#astra_SW_outputs[1].header

# turn the important columns into a pandas dataframe

astra_SW_outputs_df=pd.DataFrame({'sdss_id':astra_SW_outputs[1].data['sdss_id '].tolist(), #SDSS-5 unique identifier 
                                    'gaia_dr3_source_id':astra_SW_outputs[1].data['gaia_dr3_source_id'].tolist(), #Gaia DR3 source identifier
                                    'fieldid':astra_SW_outputs[1].data['fieldid '].tolist(), #Field identifier
                                    'mjd':astra_SW_outputs[1].data['mjd     '].tolist(), #Modified Julian date of observation 
                                    'catalogid21':astra_SW_outputs[1].data['catalogid21'].tolist(), #Catalog identifier (v21; v0.0)  
                                    'catalogid25':astra_SW_outputs[1].data['catalogid25'].tolist(), #Catalog identifier (v25; v0.5)
                                    'catalogid31':astra_SW_outputs[1].data['catalogid31'].tolist(), #Catalog identifier (v31; v1.0)
                                    'ra':astra_SW_outputs[1].data['ra      '].tolist(), #Right ascension [deg] 
                                    'dec':astra_SW_outputs[1].data['dec     '].tolist(), #Declination [deg] 
                                    'l':astra_SW_outputs[1].data['l       '].tolist(), #Galactic longitude [deg]  
                                    'b':astra_SW_outputs[1].data['b       '].tolist(), #Galactic latitude [deg]
                                    'r_med_geo':astra_SW_outputs[1].data['r_med_geo'].tolist(), #Median geometric distance [pc]
                                    'r_lo_geo':astra_SW_outputs[1].data['r_lo_geo'].tolist(), #16th percentile of geometric distance [pc]
                                    'r_hi_geo':astra_SW_outputs[1].data['r_hi_geo'].tolist(), #84th percentile of geometric distance [pc]  
                                    'bailer_jones_flags':astra_SW_outputs[1].data['bailer_jones_flags'].tolist(), #Bailer-Jones quality flags 
                                    'n_exp':astra_SW_outputs[1].data['n_exp   '].tolist(), #Number of co-added exposures 
                                    'exptime':astra_SW_outputs[1].data['exptime '].tolist(), #Exposure time [s]
                                    'snr':astra_SW_outputs[1].data['snr     '].tolist(), #Signal-to-noise ratio, SN_MEDIAN_ALL from spAll file
                                    'classification':astra_SW_outputs[1].data['classification'].tolist(), #Classification
                                    'p_da':astra_SW_outputs[1].data['p_da    '].tolist(), #DA-type white dwarf probability 
                                    'teff_snowwhite':astra_SW_outputs[1].data['teff    '].tolist(), #Stellar effective temperature [K] 
                                    'e_teff_snowwhite':astra_SW_outputs[1].data['e_teff  '].tolist(), #Error on stellar effective temperature [K]  
                                    'logg_snowwhite':astra_SW_outputs[1].data['logg    '].tolist(), #Surface gravity [log10(cm/s^2)]    
                                    'e_logg_snowwhite':astra_SW_outputs[1].data['e_logg  '].tolist(), #Error on surface gravity [log10(cm/s^2)] 
                                   })
            
display(astra_SW_outputs_df)

,sdss_id,gaia_dr3_source_id,fieldid,mjd,catalogid21,catalogid25,catalogid31,ra,dec,l,b,r_med_geo,r_lo_geo,r_hi_geo,bailer_jones_flags,n_exp,exptime,snr,classification,p_da,teff_snowwhite,e_teff_snowwhite,logg_snowwhite,e_logg_snowwhite
0,66900799,2064465331787360768,15371,59392,4208244852,27021597767779661,63050395004738987,309.097870,39.790707,79.360954,-0.602446,586.181946,491.870850,703.074707,10000,4,3600.0,2.025518,['DZ/CV'],0.031109,NaN,NaN,NaN,NaN
1,66900799,2064465331787360768,15371,59367,4208244852,27021597767779661,63050395004738987,309.097870,39.790707,79.360954,-0.602446,586.181946,491.870850,703.074707,10000,1,900.0,2.236021,DA,0.638688,13000.000000,7382.820312,7.050000,0.453849
2,66900799,2064465331787360768,15371,59387,4208244852,27021597767779661,63050395004738987,309.097870,39.790707,79.360954,-0.602446,586.181946,491.870850,703.074707,10000,5,4500.0,1.356097,DA_MS:,0.149444,NaN,NaN,NaN,NaN
3,66902678,2064629919226999552,15371,59392,4208245421,27021597767780227,63050395004761024,308.850861,39.829388,79.277802,-0.427557,459.896027,416.635376,522.768005,10000,4,3600.0,2.416768,['DA/DAZ'],0.227482,NaN,NaN,NaN,NaN
4,66902678,2064629919226999552,15371,59387,4208245421,27021597767780227,63050395004761024,308.850861,39.829388,79.277802,-0.427557,459.896027,416.635376,522.768005,10000,5,4500.0,5.239281,DA,0.581703,12419.947266,547.244141,7.845727,0.182469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26185,64008877,1836557054060949120,102417,60128,5381758805,27021598937993091,63050394895313891,302.830811,27.135986,65.997139,-3.585594,73.086708,72.353294,73.723122,10033,1,900.0,3.770883,CV,0.000032,NaN,NaN,NaN,NaN
26186,64009349,1836596838326994688,102417,60129,5381786400,27021598938020635,63050394895336634,303.004761,27.513887,66.398865,-3.508610,402.270630,345.946289,468.861847,10000,1,900.0,1.298547,['DAZ/DZA'],0.193308,NaN,NaN,NaN,NaN
26187,66218924,2029085143545889152,102417,60129,5385111919,27021598941343496,63050394970610358,301.388794,28.422886,66.390190,-1.816549,90.181610,89.246872,91.234665,10033,1,900.0,4.439527,['DABZ/DA'],0.120077,NaN,NaN,NaN,NaN
26188,66218449,2029053227658163456,102417,60128,5385130196,27021598941361751,63050394970592171,301.886749,28.459167,66.656380,-2.166203,490.957550,414.961182,576.633484,10022,1,900.0,0.481031,DA,0.405992,13000.000000,18365.472656,7.050000,1.020708


In [4]:
astra_SW_outputs_df['classification'].unique()

array(["['DZ/CV']", 'DA', 'DA_MS:', "['DA/DAZ']", 'DA:', 'DZB:', 'DB',
       'DA_MS', 'CV', 'DB_MS', "['CV/DA_MS']", "['DC/DZ']", 'DC:', 'DC',
       "['DA/DZ']", "['DA/DA_MS']", "['DZ/DBZ']", "['DZ/DC']", 'DB:',
       "['DA_MS/CV']", "['DA_MS/DA']", 'DZ:', "['DBZ/DZ']", "['DC/DA']",
       'DBAZ:', "['DZA/DZB']", "['DA/CV']", "['DZ/DZB']", "['DZA/DA']",
       "['DZ/DAZ']", "['DB_MS/DB']", "['DB/DB_MS']", 'DB_MS:',
       "['DBZ/DBAZ']", 'DZ', "['DC/DA_MS']", "['DZA/DAZ']", "['DZ/DA']",
       "['DZB/DBZ']", "['DAB/DB_MS']", "['CV/DZ']", "['DB/DC']",
       "['DC/DC_MS']", "['DA/DC']", "['DB/DAB']", 'DAO:', 'DAO', 'DZA',
       "['CV/DQpec']", "['DQpec/CV']", "['DA/DAB']", "['DBA/DABZ']",
       "['DC/DZBA']", "['CV/DC']", "['DC_MS/DA_MS']", "['DZ/DZA']",
       "['DAO/DZA']", 'DZB', "['DZB/DZ']", "['DZBA/DZ']", "['DBAZ/DZB']",
       'DQpec:', 'DQpec', "['DABZ/DBAZ']", "['DA/DB']", "['DA/DAO']",
       "['DZB/DZA']", "['hotDQ/DC']", "['DO/DAO']", "['DC/DO']",
       "['DZ/DQpec']",

# Select only DA WDs
Must have full DA classification, not selecting uncertain DAs (DA:) of which there are 638

In [5]:
SDSS_DA_df=astra_SW_outputs_df.query("classification=='DA'")
#reset the indices
indices=np.arange(0,len(SDSS_DA_df))
SDSS_DA_df.set_index(indices,inplace=True)
print(len(SDSS_DA_df))

16652


In [6]:
len(astra_SW_outputs_df.query("classification=='DA:'"))

638

# Fun with Catalogids
There are 3 different catalogids. For ipl-3, if the fieldid<16000 use catalogid21 and if >=16000 use catalogid25

In [7]:
#add column for actual catalogid found in ipl-3
catalogids=[]
for i in range(len(SDSS_DA_df)):
    if SDSS_DA_df['fieldid'][i]<16000:
        catalogids.append(SDSS_DA_df['catalogid21'][i])
    else:
        catalogids.append(SDSS_DA_df['catalogid25'][i])
SDSS_DA_df['catalogid']=catalogids

/var/folders/ny/bjybyvhj6s9_2ms5cmld1xbc0000gn/T/ipykernel_1756/3547597819.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SDSS_DA_df['catalogid']=catalogids


In [8]:
#add column for spectrum file name
spec_files=[]
for ind in range(len(SDSS_DA_df)):
    fieldid=str(SDSS_DA_df['fieldid'][ind])
    if len(fieldid)<6:
        fieldid='0'+fieldid
    mjd=str(SDSS_DA_df['mjd'][ind])
    catalogid=str(SDSS_DA_df['catalogid'][ind])
    fitsfilename='spec-'+fieldid+'-'+mjd+'-'+catalogid+'.fits'
    spec_files.append(fitsfilename)
SDSS_DA_df['spec_file']=spec_files

/var/folders/ny/bjybyvhj6s9_2ms5cmld1xbc0000gn/T/ipykernel_1756/1555076035.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SDSS_DA_df['spec_file']=spec_files


# Get the Gaia Photometry

https://www2.mpia-hd.mpg.de/~calj/gedr3_distances.html
- The Bailer-Jones source id is the same as in EDR3
- r med geo, r lo geo, and r hi geo are the median, 16th percentile, and 84th percentile of the geometric distance posterior in parsec

https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html
- phot_g_mean_flux : G-band mean flux (double, Flux[e-/s]), Mean flux in the G-band. 
- phot_g_mean_flux_error : Error on G-band mean flux (double, Flux[e-/s]), Standard deviation of the G-band fluxes divided by sqrt(phot_g_n_obs) 
- phot_g_mean_mag : G-band mean magnitude (float, Magnitude[mag]), Mean magnitude in the G band. This is computed from the G-band mean flux applying the magnitude zero-point in the Vega scale. No error is provided for this quantity as the error distribution is only symmetric in flux space. This converts to an asymmetric error distribution in magnitude space which cannot be represented by a single error value. 
- Have same params as above for BP and RP bands
- phot_bp_rp_excess_factor : BP/RP excess factor (float), BP/RP excess factor estimated from the comparison of the sum of integrated BP and RP fluxes with respect to the flux in the G band. This measures the excess of flux in the BP and RP integrated photometry with respect to the G band. This excess is believed to be caused by background and contamination issues affecting the BP and RP data. Therefore a large value of this factor for a given source indicates systematic errors in the BP and RP photometry.
- bp_rp : BP - RP colour (float, Magnitude[mag]), BP - RP colour: phot_bp_mean_mag - phot_rp_mean_mag 
- bp_g : BP - G colour (float, Magnitude[mag]), BP - G colour: phot_bp_mean_mag - phot_g_mean_mag 
- g_rp : G - RP colour (float, Magnitude[mag]), G - RP colour: phot_g_mean_mag - phot_rp_mean_mag 
- l : Galactic longitude (double, Angle[deg]), Galactic Longitude of the object at reference epoch ref_epoch, see Section 3.1.7 of the release documentation for conversion details. 
- b : Galactic latitude (double, Angle[deg]), Galactic Latitude of the object at reference epoch ref_epoch, see Section 3.1.7 of the release documentation for conversion details. 
- - pmra : Proper motion in right ascension direction (double, Angular Velocity[mas/year]). Proper motion in right ascension of the source in ICRS at the reference epoch ref_epoch. This is the local tangent plane projection of the proper motion vector in the direction of increasing right ascension. 
- pmra_error : Standard error of proper motion in right ascension direction (double, Angular Velocity[mas/year] ). 
- pmdec : Proper motion in declination direction (double, Angular Velocity[mas/year] ). Proper motion in declination of the source at the reference epoch ref_epoch. This is the projection of the proper motion vector in the direction of increasing declination.
- pmdec_error : Standard error of proper motion in declination direction (double, Angular Velocity[mas/year] )

In [9]:
def get_gaia_info(wd_table):
    stardats = []
    id_list=tuple(wd_table['gaia_dr3_source_id'].unique())
    iters = (len(id_list)+100) // 100

    #query gaia to get the photometry for each object
    for i in tqdm(range(iters)):
        ADQL_CODE1 = """SELECT dist.source_id, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.phot_g_mean_flux, gaia.phot_g_mean_flux_error, gaia.phot_g_mean_mag, gaia.phot_bp_mean_flux, gaia.phot_bp_mean_flux_error, gaia.phot_bp_mean_mag, gaia.phot_rp_mean_flux, gaia.phot_rp_mean_flux_error, gaia.phot_rp_mean_mag, gaia.phot_bp_rp_excess_factor 
        FROM gaiadr3.gaia_source as gaia
        JOIN external.gaiaedr3_distance as dist
        ON gaia.source_id = dist.source_id      
        WHERE gaia.source_id in {}""".format(id_list[100*i:100*i+100])
        stardats.append(Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())
    
    gaia_astpy = vstack(stardats)
    gaia_astpy.rename_column('source_id', 'gaia_dr3_source_id')
    gaia_df=gaia_astpy.to_pandas()
    
    catalog=pd.merge(left=wd_table,right=gaia_df,on='gaia_dr3_source_id',how="inner")
    
    #check to make sure the gaia BP+RP photometry (these are what we will use to fit later)
    #and distances exist for each object
    nophotbp=np.isnan(catalog['phot_bp_mean_flux'].to_list())
    nophotrp=np.isnan(catalog['phot_rp_mean_flux'].to_list())
    nodist=np.isnan(catalog['r_med_geo'].to_list())
    nogaiaphot=[]
    for i in range(len(nophotbp)):
        if nophotbp[i]==False and nophotrp[i]==False:
            nogaiaphot.append(False)
        else:
            nogaiaphot.append(True)
    catalog['no_distance']=nodist
    catalog['no_gaia_phot']=nogaiaphot
    return(catalog,nophotbp,nophotrp,nodist)

In [10]:
SDSS_DA_df,nophotbp,nophotrp,nodist=get_gaia_info(SDSS_DA_df)

100%|███████████████████████████████████████████| 86/86 [01:08<00:00,  1.26it/s]


# Get the SDSS Photometry

The SDSS imaging camera was retired in 2009. So, there is no new SDSS photometry available, only new reductions of old photometric data.

- https://en.wikipedia.org/wiki/Sloan_Digital_Sky_Survey
- https://www.sdss.org/instruments/

Use astroquery to get the photometry given the coordinates of each object.

A full list of all photoobj_fields that can be queried, along with a description of each field, can be found here
https://github.com/astropy/astroquery/blob/main/astroquery/sdss/data/PhotoObjAll_dr12.json
- Download the table and read in the data

Here is a description of the differrent SDSS measures of flux and magnitude:
https://live-sdss4org-dr12.pantheonsite.io/algorithms/magnitudes/
- Isolated stars are best described by PSF magnitudes

The columns we care about are:
- clean: Clean photometry flag (1=clean, 0=unclean) (see https://www.sdss4.org/dr17/tutorials/flags/)
- probPSF: Probability that the object is a star. Currently 0 if type == 3 (galaxy), 1 if type == 6 (star).
- flags: Photo Object Attribute Flags
- psfMag_PASSBAND: PASSBAND={u,g,r,i,z}, PSF magnitude
- psfMagErr_PASSBAND: PASSBAND={u,g,r,i,z}, PSF magnitude error
- psfFlux_PASSBAND: PASSBAND={u,g,r,i,z}, PSF flux in nanomaggies
- psfFluxIvar_PASSBAND: PASSBAND={u,g,r,i,z}, PSF flux inverse variance in nanomaggies^{-2}


In [11]:
def get_sdss_info(wd_table):
    #get set of unique coordinates
    sdss_phot_df=wd_table[['gaia_dr3_source_id','ra','dec']].drop_duplicates(subset=['ra','dec'])
    coords = SkyCoord(sdss_phot_df['ra'], sdss_phot_df['dec'], unit='deg')

    stardats = []
    phot_inds=[]

    #query sdss to get the photometry for each object
    photoobj_fields_list=['clean','probPSF',
             'psfMag_u','psfMag_g','psfMag_r','psfMag_i','psfMag_z',
             'psfMagErr_u','psfMagErr_g','psfMagErr_r','psfMagErr_i','psfMagErr_z',
             'psfFlux_u','psfFlux_g','psfFlux_r','psfFlux_i','psfFlux_z',
             'psfFluxIvar_u','psfFluxIvar_g','psfFluxIvar_r','psfFluxIvar_i','psfFluxIvar_z']
    clean=[]
    probPSF=[]
    psfMag_u=[]
    psfMag_g=[]
    psfMag_r=[]
    psfMag_i=[]
    psfMag_z=[]
    psfMagErr_u=[]
    psfMagErr_g=[]
    psfMagErr_r=[]
    psfMagErr_i=[]
    psfMagErr_z=[]
    psfFlux_u=[]
    psfFlux_g=[]
    psfFlux_r=[]
    psfFlux_i=[]
    psfFlux_z=[]
    psfFluxIvar_u=[]
    psfFluxIvar_g=[]
    psfFluxIvar_r=[]
    psfFluxIvar_i=[]
    psfFluxIvar_z=[]
    for coord in tqdm(coords):
        try:
            SDSS_query = SDSS.query_crossid(coord, photoobj_fields=photoobj_fields_list)
            clean.append(SDSS_query['clean'][0])
            probPSF.append(SDSS_query['probPSF'][0])
            psfMag_u.append(SDSS_query['psfMag_u'][0])
            psfMag_g.append(SDSS_query['psfMag_g'][0])
            psfMag_r.append(SDSS_query['psfMag_r'][0])
            psfMag_i.append(SDSS_query['psfMag_i'][0])
            psfMag_z.append(SDSS_query['psfMag_z'][0])
            psfMagErr_u.append(SDSS_query['psfMagErr_u'][0])
            psfMagErr_g.append(SDSS_query['psfMagErr_g'][0])
            psfMagErr_r.append(SDSS_query['psfMagErr_r'][0])
            psfMagErr_i.append(SDSS_query['psfMagErr_i'][0])
            psfMagErr_z.append(SDSS_query['psfMagErr_z'][0])
            psfFlux_u.append(SDSS_query['psfFlux_u'][0])
            psfFlux_g.append(SDSS_query['psfFlux_g'][0])
            psfFlux_r.append(SDSS_query['psfFlux_r'][0])
            psfFlux_i.append(SDSS_query['psfFlux_i'][0])
            psfFlux_z.append(SDSS_query['psfFlux_z'][0])
            psfFluxIvar_u.append(SDSS_query['psfFluxIvar_u'][0])
            psfFluxIvar_g.append(SDSS_query['psfFluxIvar_g'][0])
            psfFluxIvar_r.append(SDSS_query['psfFluxIvar_r'][0])
            psfFluxIvar_i.append(SDSS_query['psfFluxIvar_i'][0])
            psfFluxIvar_z.append(SDSS_query['psfFluxIvar_z'][0])
        except:
            #if no SDSS photometry, fill with np.nan
            clean.append(np.nan)
            probPSF.append(np.nan)
            psfMag_u.append(np.nan)
            psfMag_g.append(np.nan)
            psfMag_r.append(np.nan)
            psfMag_i.append(np.nan)
            psfMag_z.append(np.nan)
            psfMagErr_u.append(np.nan)
            psfMagErr_g.append(np.nan)
            psfMagErr_r.append(np.nan)
            psfMagErr_i.append(np.nan)
            psfMagErr_z.append(np.nan)
            psfFlux_u.append(np.nan)
            psfFlux_g.append(np.nan)
            psfFlux_r.append(np.nan)
            psfFlux_i.append(np.nan)
            psfFlux_z.append(np.nan)
            psfFluxIvar_u.append(np.nan)
            psfFluxIvar_g.append(np.nan)
            psfFluxIvar_r.append(np.nan)
            psfFluxIvar_i.append(np.nan)
            psfFluxIvar_z.append(np.nan)
    sdss_phot_df['clean']=clean
    sdss_phot_df['prob_psf']=probPSF
    sdss_phot_df['psf_mag_u']=psfMag_u
    sdss_phot_df['psf_mag_g']=psfMag_g
    sdss_phot_df['psf_mag_r']=psfMag_r
    sdss_phot_df['psf_mag_i']=psfMag_i
    sdss_phot_df['psf_mag_z']=psfMag_z
    sdss_phot_df['psf_magerr_u']=psfMagErr_u
    sdss_phot_df['psf_magerr_g']=psfMagErr_g
    sdss_phot_df['psf_magerr_r']=psfMagErr_r
    sdss_phot_df['psf_magerr_i']=psfMagErr_i
    sdss_phot_df['psf_magerr_z']=psfMagErr_z
    sdss_phot_df['psf_flux_u']=psfFlux_u
    sdss_phot_df['psf_flux_g']=psfFlux_g
    sdss_phot_df['psf_flux_r']=psfFlux_r
    sdss_phot_df['psf_flux_i']=psfFlux_i
    sdss_phot_df['psf_flux_z']=psfFlux_z
    sdss_phot_df['psf_fluxivar_u']=psfFluxIvar_u
    sdss_phot_df['psf_fluxivar_g']=psfFluxIvar_g
    sdss_phot_df['psf_fluxivar_r']=psfFluxIvar_r
    sdss_phot_df['psf_fluxivar_i']=psfFluxIvar_i
    sdss_phot_df['psf_fluxivar_z']=psfFluxIvar_z

    sdss_phot_df=sdss_phot_df.drop(['ra','dec'],axis=1)

    catalog=pd.merge(left=wd_table,right=sdss_phot_df,on='gaia_dr3_source_id',how="inner")
    
    return(catalog)

In [12]:
SDSS_DA_df=get_sdss_info(SDSS_DA_df)

  0%|                                                  | 0/8546 [00:00<?, ?it/s]/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|█████████████████████████████████████| 8546/8546 [00:08<00:00, 1054.92it/s]


### Convert SDSS magnitudes to the AB system

The SDSS flux is in weird units (nanomaggies) and the magnitude system is nearly the AB system but not quite and is also in asinh magnitudes instead of normal magnitudes. (see Conversion from SDSS ugriz magnitudes to AB ugriz magnitudes at https://live-sdss4org-dr12.pantheonsite.io/algorithms/fluxcal/#SDSStoAB, also see http://faraday.uwyo.edu/~admyers/ASTR5160/handouts/516016.pdf). So first we must convert the fluxes to normal magnitudes on the SDSS system
$$m=-2.5\text{log}_{10}(f/f_0)$$
where the zero-point is $3631\times10^9\text{ Jy}=10^9 f_0$, plugging into the expression for $m$, if $f$ is in nanomaggies then the magnitude is
$$m=22.5-2.5\text{log}_{10}(f)$$

Then apply the corrections to the AB system from https://iopscience.iop.org/article/10.1086/507110
$$u_{AB}=u_{SDSS}-0.040\text{ mag}$$
$$g_{AB}=g_{SDSS}\text{ mag}$$
$$r_{AB}=r_{SDSS}\text{ mag}$$
$$i_{AB}=i_{SDSS}+0.015\text{ mag}$$
$$z_{AB}=z_{SDSS}+0.030\text{ mag}$$

These statements are not precise to better than 0.01 mag, so add that error in quadrature.

In [13]:
SDSS_DA_df['mag_ab_u']=22.5-2.5*np.log10(SDSS_DA_df['psf_flux_u'])-0.04
SDSS_DA_df['magerr_ab_u']=np.sqrt(SDSS_DA_df['psf_magerr_u']**2+0.01**2)

SDSS_DA_df['mag_ab_g']=22.5-2.5*np.log10(SDSS_DA_df['psf_flux_g'])
SDSS_DA_df['magerr_ab_g']=np.sqrt(SDSS_DA_df['psf_magerr_g']**2+0.01**2)

SDSS_DA_df['mag_ab_r']=22.5-2.5*np.log10(SDSS_DA_df['psf_flux_r'])
SDSS_DA_df['magerr_ab_r']=np.sqrt(SDSS_DA_df['psf_magerr_r']**2+0.01**2)

SDSS_DA_df['mag_ab_i']=22.5-2.5*np.log10(SDSS_DA_df['psf_flux_i'])+0.015
SDSS_DA_df['magerr_ab_i']=np.sqrt(SDSS_DA_df['psf_magerr_i']**2+0.01**2)

SDSS_DA_df['mag_ab_z']=22.5-2.5*np.log10(SDSS_DA_df['psf_flux_z'])+0.03
SDSS_DA_df['magerr_ab_z']=np.sqrt(SDSS_DA_df['psf_magerr_z']**2+0.01**2)

/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  resu

In [14]:
#for photometric fitting, we will use SDSS u+r+z bands 
#flag any object that is missing SDSS photometry or has negative fluxes
nophotmag_u=np.isnan(SDSS_DA_df['mag_ab_u'].to_list())
nophotmag_r=np.isnan(SDSS_DA_df['mag_ab_r'].to_list())
nophotmag_z=np.isnan(SDSS_DA_df['mag_ab_z'].to_list())
nophotmagerr_u=np.isnan(SDSS_DA_df['magerr_ab_u'].to_list())
nophotmagerr_r=np.isnan(SDSS_DA_df['magerr_ab_r'].to_list())
nophotmagerr_z=np.isnan(SDSS_DA_df['magerr_ab_z'].to_list())

#flag any object with 0 SDSS flux (infinite magnitude)
zero_u=np.isnan(SDSS_DA_df['mag_ab_u'].where(SDSS_DA_df['mag_ab_u']<np.inf).to_list())
zero_r=np.isnan(SDSS_DA_df['mag_ab_r'].where(SDSS_DA_df['mag_ab_r']<np.inf).to_list())
zero_z=np.isnan(SDSS_DA_df['mag_ab_z'].where(SDSS_DA_df['mag_ab_z']<np.inf).to_list())

nosdssphot=[]
for i in range(len(nophotmag_u)):
    if nophotmag_u[i]==False and nophotmag_r[i]==False and nophotmag_z[i]==False and nophotmagerr_u[i]==False and nophotmagerr_r[i]==False and nophotmagerr_z[i]==False and zero_u[i]==False and zero_r[i]==False and zero_z[i]==False:
        nosdssphot.append(False)
    else:
        nosdssphot.append(True)
SDSS_DA_df['no_sdss_phot']=nosdssphot


# Check that all objects
- have either clean SDSS or clean Gaia photometry
- have Bailer-Jones distances

Remove the one object with neither clean SDSS nor clean Gaia photometry

In [15]:
display(SDSS_DA_df.query('(no_sdss_phot==True or clean==0) and (no_gaia_phot==True or phot_bp_rp_excess_factor>2)'))

#remove this object
bad_ind=SDSS_DA_df.query('(no_sdss_phot==True or clean==0) and (no_gaia_phot==True or phot_bp_rp_excess_factor>2)').index
SDSS_DA_df=SDSS_DA_df.drop(bad_ind)
#reset the indices
indices=np.arange(0,len(SDSS_DA_df))
SDSS_DA_df.set_index(indices,inplace=True)
    

,sdss_id,gaia_dr3_source_id,fieldid,mjd,catalogid21,catalogid25,catalogid31,ra,dec,l,b,r_med_geo,r_lo_geo,r_hi_geo,bailer_jones_flags,n_exp,exptime,snr,classification,p_da,teff_snowwhite,e_teff_snowwhite,logg_snowwhite,e_logg_snowwhite,catalogid,spec_file,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,no_distance,no_gaia_phot,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot
16319,86743540,4249619656907184768,100993,59840,5315946288,27021598872293136,63050395598997238,304.302124,6.635192,49.120869,-15.693171,319.49765,308.224823,334.679138,10122,1,900.0,13.934942,DA,0.74638,9613.09668,1858.192993,7.419135,0.162643,27021598872293136,spec-100993-59840-27021598872293136.fits,7.384789,0.174438,-5.260786,0.147804,835.950139,1.806449,18.381916,1047.01896,90.456047,17.788656,720.41184,70.154449,17.603943,2.114278,False,False,0.0,1.0,18.45111,18.31273,19.04965,24.36195,22.82677,0.032585,0.135166,0.846656,41.02299,14.38003,41.644,47.30488,23.99546,-0.000043,0.000199,0.640173,0.028833,0.002856,0.005405,0.002603,18.411119,0.034085,18.312735,0.135536,19.049677,0.846715,NaN,41.022991,31.78331,14.380033,False


In [16]:
display(SDSS_DA_df.query('no_distance==True'))

,sdss_id,gaia_dr3_source_id,fieldid,mjd,catalogid21,catalogid25,catalogid31,ra,dec,l,b,r_med_geo,r_lo_geo,r_hi_geo,bailer_jones_flags,n_exp,exptime,snr,classification,p_da,teff_snowwhite,e_teff_snowwhite,logg_snowwhite,e_logg_snowwhite,catalogid,spec_file,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,no_distance,no_gaia_phot,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot


### Number of unique WDs in SDSS-V: 8545

In [17]:
len(SDSS_DA_df['gaia_dr3_source_id'].unique())

8545

# Save the table of SDSS-V DA WDs as a .csv file and import back in if needed

In [18]:
#save to backup
SDSS_DA_df.to_csv('csv/SDSSV_DA_df_nb00.csv',index=False)

In [19]:
#save overall result
SDSS_DA_df.to_csv('csv/SDSSV_DA_df.csv',index=False)


In [20]:
#import back in the data, either from the backup file or from the latest version of the table
backup=False
if backup==True:
    SDSS_DA_df=pd.read_csv('csv/SDSSV_DA_df_nb00.csv')
else:
    SDSS_DA_df=pd.read_csv('csv/SDSSV_DA_df.csv')
display(SDSS_DA_df)

,sdss_id,gaia_dr3_source_id,fieldid,mjd,catalogid21,catalogid25,catalogid31,ra,dec,l,b,r_med_geo,r_lo_geo,r_hi_geo,bailer_jones_flags,n_exp,exptime,snr,classification,p_da,teff_snowwhite,e_teff_snowwhite,logg_snowwhite,e_logg_snowwhite,catalogid,spec_file,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,no_distance,no_gaia_phot,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot
0,66900799,2064465331787360768,15371,59367,4208244852,27021597767779661,63050395004738987,309.097870,39.790707,79.360954,-0.602446,586.181946,491.870850,703.074707,10000,1,900.0,2.236021,DA,0.638688,13000.000000,7382.820312,7.050000,0.453849,4208244852,spec-015371-59367-4208244852.fits,6.234600,0.310137,7.095345,0.357636,302.009777,0.955366,19.487314,228.468159,7.704186,19.441479,113.633559,7.549196,19.609129,1.132750,False,False,1.0,1.0,19.36022,19.32954,19.63350,19.96624,20.26875,0.027439,0.019495,0.018669,0.028313,0.124330,18.02547,18.54267,14.01424,10.31272,7.737120,4.817704,9.019837,17.215990,13.810460,1.228934,19.320284,0.029204,19.329569,0.021911,19.633576,0.021179,19.981567,0.030027,20.308552,0.124732,False
1,66902678,2064629919226999552,15371,59387,4208245421,27021597767780227,63050395004761024,308.850861,39.829388,79.277802,-0.427557,459.896027,416.635376,522.768005,10000,5,4500.0,5.239281,DA,0.581703,12419.947266,547.244141,7.845727,0.182469,4208245421,spec-015371-59387-4208245421.fits,-2.528749,0.325960,-10.725447,0.315941,276.473537,0.844132,19.583233,193.603408,7.463188,19.621260,98.636197,5.550840,19.762804,1.057026,False,False,1.0,1.0,19.59193,19.43115,19.76566,19.99559,20.29769,0.031457,0.020564,0.024144,0.036363,0.132072,14.56094,16.88607,12.40784,10.03745,7.529879,5.616520,9.774905,13.130340,8.837507,1.147590,19.552026,0.033008,19.431179,0.022867,19.765760,0.026133,20.010942,0.037713,20.338030,0.132450,False
2,66902096,2064573324440163456,15371,59367,4208256829,27021597767791601,63050395004755297,309.594055,40.293499,79.991142,-0.601126,423.614502,405.111725,445.662018,10000,1,900.0,5.704046,DA,0.687817,22541.693359,3095.440674,8.823078,0.225764,4208256829,spec-015371-59367-4208256829.fits,14.470780,0.103633,3.542376,0.133135,993.572640,1.184968,18.194368,803.955601,9.695243,18.075462,317.644781,7.352390,18.493042,1.128856,False,False,1.0,1.0,17.67537,17.98490,18.39462,18.76124,19.07410,0.020611,0.019703,0.015666,0.021490,0.043710,85.08470,63.97914,43.86797,31.29605,23.438430,0.383308,0.741824,2.495894,2.605837,1.118684,17.635371,0.022909,17.984904,0.022095,18.394631,0.018586,18.776276,0.023703,19.105179,0.044839,False
3,66902096,2064573324440163456,15371,59387,4208256829,27021597767791601,63050395004755297,309.594055,40.293499,79.991142,-0.601126,423.614502,405.111725,445.662018,10000,5,4500.0,16.457048,DA,0.888221,30454.421875,734.626465,9.490000,0.116178,4208256829,spec-015371-59387-4208256829.fits,14.470780,0.103633,3.542376,0.133135,993.572640,1.184968,18.194368,803.955601,9.695243,18.075462,317.644781,7.352390,18.493042,1.128856,False,False,1.0,1.0,17.67537,17.98490,18.39462,18.76124,19.07410,0.020611,0.019703,0.015666,0.021490,0.043710,85.08470,63.97914,43.86797,31.29605,23.438430,0.383308,0.741824,2.495894,2.605837,1.118684,17.635371,0.022909,17.984904,0.022095,18.394631,0.018586,18.776276,0.023703,19.105179,0.044839,False
4,66902096,2064573324440163456,15371,59392,4208256829,27021597767791601,63050395004755297,309.594055,40.293499,79.991142,-0.601126,423.614502,405.111725,445.662018,10000,4,3600.0,7.459354,DA,0.701336,27087.